In [299]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from rdkit import Chem
from rdkit.Chem import Descriptors, SaltRemover, MolFromSmiles, Draw, GetFormalCharge, MolToSmiles
from rdkit.Chem.MolStandardize import rdMolStandardize 
from statistics import mean

### Dataset 1 (https://github.com/theochem/B3DB)

In [300]:
data_1 = pd.read_csv('B3DB_classification.tsv', sep='\t')
data_1.head()

,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN


In [301]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7807 entries, 0 to 7806
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NO.            7807 non-null   int64  
 1   compound_name  6698 non-null   object 
 2   IUPAC_name     6170 non-null   object 
 3   SMILES         7807 non-null   object 
 4   CID            6170 non-null   float64
 5   logBB          1058 non-null   float64
 6   BBB+/BBB-      7807 non-null   object 
 7   Inchi          7807 non-null   object 
 8   threshold      3621 non-null   float64
 9   reference      7807 non-null   object 
 10  group          7807 non-null   object 
 11  comments       18 non-null     object 
dtypes: float64(3), int64(1), object(8)
memory usage: 732.0+ KB


In [302]:
data_1 = data_1[['SMILES', 'BBB+/BBB-']]


In [303]:
data_1['SMILES'] = data_1['SMILES'].apply(lambda smi: Chem.CanonSmiles(Chem.MolToSmiles(Chem.MolFromSmiles(smi))))

In [304]:
data_1.head()

,SMILES,BBB+/BBB-
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,BBB-
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,BBB-
2,Oc1c(I)cc(Cl)c2cccnc12,BBB-
3,CCNC(=NCCSCc1ncccc1Br)NC#N,BBB-
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,BBB-


In [305]:
data_1['BBB+/BBB-'] = data_1['BBB+/BBB-'].apply(lambda x: 1 if x == 'BBB+' else 0)
data_1.head()

,SMILES,BBB+/BBB-
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,0
2,Oc1c(I)cc(Cl)c2cccnc12,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,0


In [306]:
data_1 = data_1.rename(columns={'SMILES':'SMILES', 'BBB+/BBB-':'Activity'})
data_1.head()

,SMILES,Activity
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,0
2,Oc1c(I)cc(Cl)c2cccnc12,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,0


In [307]:
data_1['Activity'].value_counts()

1    4956
0    2851
Name: Activity, dtype: int64

In [308]:
data_1.drop_duplicates()

,SMILES,Activity
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,0
2,Oc1c(I)cc(Cl)c2cccnc12,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,0
...,...,...
7802,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,0
7803,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,1
7804,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,1
7805,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,0


In [309]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7807 entries, 0 to 7806
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    7807 non-null   object
 1   Activity  7807 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 122.1+ KB


In [310]:
for i, row in data_1.iterrows():
    if 'C' not in row.SMILES and 'c' not in row.SMILES:
        data_1 = data_1.drop(labels=[i], axis=0)
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7795 entries, 0 to 7806
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    7795 non-null   object
 1   Activity  7795 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 182.7+ KB


In [311]:
df = data_1

### SMILES Preprocessing

In [312]:
df['Agglomeration'] = df['SMILES'].apply(lambda x: '.' in x)
df['SMILES_clear'] = df['SMILES'].apply(lambda x: max(x.split('.'), key=len))
df['ROMol'] = df['SMILES_clear'].apply(lambda x: Chem.MolFromSmiles(x))
print(df.shape)

(7795, 5)


In [313]:
df = df[~df["ROMol"].isna()]
print(df.shape)

(7795, 5)


In [314]:
un = rdMolStandardize.Uncharger()

In [315]:
try:
    df['FORMAL_CHARGE_ch'] = df['SMILES_clear'].apply(lambda x: GetFormalCharge(Chem.MolFromSmiles(x)))
    df['SMILES_uncharge'] = df['ROMol'].apply(lambda x: Chem.MolToSmiles(un.uncharge(x), kekuleSmiles=True))
    df['FORMAL_CHARGE_unch'] = df['SMILES_uncharge'].apply(lambda x: GetFormalCharge(Chem.MolFromSmiles(x)))
except:
    pass


[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Run

[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Run

[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Running Uncharger
[21:59:04] Run

[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Run

[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Run

[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Run

[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Running Uncharger
[21:59:05] Run

In [316]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7795 entries, 0 to 7806
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SMILES              7795 non-null   object
 1   Activity            7795 non-null   int64 
 2   Agglomeration       7795 non-null   bool  
 3   SMILES_clear        7795 non-null   object
 4   ROMol               7795 non-null   object
 5   FORMAL_CHARGE_ch    7795 non-null   int64 
 6   SMILES_uncharge     7795 non-null   object
 7   FORMAL_CHARGE_unch  7795 non-null   int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 494.8+ KB


In [317]:
#df.drop(labels=['SMILES_clear', 'ROMol', 'FORMAL_CHARGE_ch'], inplace=True, axis=1)


In [318]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7795 entries, 0 to 7806
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SMILES              7795 non-null   object
 1   Activity            7795 non-null   int64 
 2   Agglomeration       7795 non-null   bool  
 3   SMILES_clear        7795 non-null   object
 4   ROMol               7795 non-null   object
 5   FORMAL_CHARGE_ch    7795 non-null   int64 
 6   SMILES_uncharge     7795 non-null   object
 7   FORMAL_CHARGE_unch  7795 non-null   int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 494.8+ KB


In [319]:
df.reset_index(drop=True, inplace=True)
df.head()

,SMILES,Activity,Agglomeration,SMILES_clear,ROMol,FORMAL_CHARGE_ch,SMILES_uncharge,FORMAL_CHARGE_unch
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,0,False,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,<rdkit.Chem.rdchem.Mol object at 0x0000019A3A3...,0,O=C(O)C1=CC(N=NC2=CC=C(S(=O)(=O)NC3=CC=CC=N3)C...,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,0,False,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,COC1(NC(=O)C(C(=O)O)C2=CC=C(O)C=C2)C(=O)N2C(C(...,0
2,Oc1c(I)cc(Cl)c2cccnc12,0,False,Oc1c(I)cc(Cl)c2cccnc12,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,OC1=C(I)C=C(Cl)C2=CC=CN=C12,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,0,False,CCNC(=NCCSCc1ncccc1Br)NC#N,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,CCNC(=NCCSCC1=NC=CC=C1Br)NC#N,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,0,False,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,CN1CC[C@]23C4=C5C=CC(OC6O[C@H](C(=O)O)[C@@H](O...,0


### Remove inorganic

In [320]:
index_sm = df.index.tolist()

In [321]:
print(index_sm[:5])

[0, 1, 2, 3, 4]


In [322]:
sm = df['SMILES_uncharge'].values
sm

array(['O=C(O)C1=CC(N=NC2=CC=C(S(=O)(=O)NC3=CC=CC=N3)C=C2)=CC=C1O',
       'COC1(NC(=O)C(C(=O)O)C2=CC=C(O)C=C2)C(=O)N2C(C(=O)O)=C(CSC3=NN=NN3C)COC21',
       'OC1=C(I)C=C(Cl)C2=CC=CN=C12', ...,
       'CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1',
       'CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1', 'NC1=CC(C2=CC=NC=C2)=CNC1=O'],
      dtype=object)

In [323]:
cl = df[df['SMILES_uncharge'] == '[O-][Cl+3]([O-])([O-])O']
cl

,SMILES,Activity,Agglomeration,SMILES_clear,ROMol,FORMAL_CHARGE_ch,SMILES_uncharge,FORMAL_CHARGE_unch


In [324]:
list_inorg_el = ['Al', 'Au', 'Ar,''Ba', 'Be', 'Bi', 'Ca','Cd', 'Co','Cr','Cu','Fe', 'Gd','Ge', 'Hf',
                 'Hg', 'In', 'K', 'Kr' 'La', 'Mg', 'Mn', 'Na', 'Ni', 'Pb', 'Pt', 'Sb', 'Sn','Sr','Te',
                 'V','Zn', 'Li', 'Xe', 'Rn', 'Ne']
index_drop = []
uniq_el = set([])
for sm_i in range(len(sm)):
    mol_sm = MolFromSmiles(sm[sm_i])
    try:
        list_at = [str(x.GetSymbol()) for x in mol_sm.GetAtoms()]
        intersec = list(set(list_at) & set(list_inorg_el))
        T_C = 'C' in list_at
        if T_C == False:
            index_drop.append(index_sm[sm_i])
            print('smiles', sm[sm_i])
        elif len(intersec) > 0:
            index_drop.append(index_sm[sm_i])
            print('intersection', intersec)
        uniq_el = uniq_el | set(list_at)
    except:
         index_drop.append(index_sm[sm_i])
   
        

In [325]:
uniq_el

{'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'Si'}

In [326]:
index_drop

[]

In [327]:
len(index_drop)

0

In [328]:
df.drop(index=index_drop, inplace=True)

In [329]:
df.describe()

,Activity,FORMAL_CHARGE_ch,FORMAL_CHARGE_unch
count,7795.000000,7795.000000,7795.000000
mean,0.634253,0.017960,0.017960
std,0.481670,0.154268,0.154268
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000
max,1.000000,3.000000,3.000000


In [330]:
len(df.duplicated(subset=['SMILES_uncharge'], keep=False))

7795

In [331]:
df_dupl = df[df.duplicated(subset=['SMILES_uncharge'], keep=False)]
df_dupl = df_dupl.sort_values(by=['SMILES_uncharge'])
df_dupl.to_csv('BBB_dupl.csv', index=False)
df_dupl

,SMILES,Activity,Agglomeration,SMILES_clear,ROMol,FORMAL_CHARGE_ch,SMILES_uncharge,FORMAL_CHARGE_unch
1349,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=...,0,False,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=...,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)C3=CC=CC=C3)...,0
1350,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=...,0,True,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=...,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)C3=CC=CC=C3)...,0
1921,CNO.Cc1oc(C(C)(C)N(C(C)(C)N)C(C)(C)N)c(C)c1C,1,True,Cc1oc(C(C)(C)N(C(C)(C)N)C(C)(C)N)c(C)c1C,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CC1=C(C)C(C)=C(C(C)(C)N(C(C)(C)N)C(C)(C)N)O1,0
3621,Cc1oc(C(C)(C)N(C(C)(C)N)C(C)(C)N)c(C)c1C,1,False,Cc1oc(C(C)(C)N(C(C)(C)N)C(C)(C)N)c(C)c1C,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CC1=C(C)C(C)=C(C(C)(C)N(C(C)(C)N)C(C)(C)N)O1,0
5527,ClCC1CO1.NCCNCCNCCNCCN,0,True,NCCNCCNCCNCCN,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,NCCNCCNCCNCCN,0
5890,NCCNCCNCCNCCN,0,False,NCCNCCNCCNCCN,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,NCCNCCNCCNCCN,0


In [332]:
df

,SMILES,Activity,Agglomeration,SMILES_clear,ROMol,FORMAL_CHARGE_ch,SMILES_uncharge,FORMAL_CHARGE_unch
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,0,False,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,<rdkit.Chem.rdchem.Mol object at 0x0000019A3A3...,0,O=C(O)C1=CC(N=NC2=CC=C(S(=O)(=O)NC3=CC=CC=N3)C...,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,0,False,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,COC1(NC(=O)C(C(=O)O)C2=CC=C(O)C=C2)C(=O)N2C(C(...,0
2,Oc1c(I)cc(Cl)c2cccnc12,0,False,Oc1c(I)cc(Cl)c2cccnc12,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,OC1=C(I)C=C(Cl)C2=CC=CN=C12,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,0,False,CCNC(=NCCSCc1ncccc1Br)NC#N,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,CCNC(=NCCSCC1=NC=CC=C1Br)NC#N,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,0,False,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,<rdkit.Chem.rdchem.Mol object at 0x0000019A659...,0,CN1CC[C@]23C4=C5C=CC(OC6O[C@H](C(=O)O)[C@@H](O...,0
...,...,...,...,...,...,...,...,...
7790,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,0,False,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,C1=CC=C(CN(CC2=NCCN2)C2=CC=CC=C2)C=C1,0
7791,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,1,False,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CCOCCN1C(N2CCCN(C)CC2)=NC2=CC=CC=C21,0
7792,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,1,False,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1,0
7793,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,0,False,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,<rdkit.Chem.rdchem.Mol object at 0x0000019A65A...,0,CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1,0


In [333]:
df = df.reset_index(drop=True)

In [334]:
df.to_csv('classification_dataset.csv',index=False)